In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
# things we need for Tensorflow
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import tensorflow as tf
import pandas as pd
import pickle
import random
import json

In [2]:
intents = json.loads(open('intents.json').read())

words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique stemmed words", words)

96 documents
22 classes ['God', 'about me', 'abuse', 'age', 'alive', 'bookings', 'creator', 'fillers', 'goodbye', 'google', 'greeting', 'joke', 'killing', 'lenny', 'myself', 'news', 'stories', 'talk', 'tasks', 'thanks', 'weather', 'wikipedia']
116 unique stemmed words ['!', "'s", ',', 'a', 'abl', 'about', 'ag', 'ai', 'al', 'alright', 'am', 'an', 'anyon', 'anyth', 'ar', 'ass', 'at', 'beat', 'bitch', 'book', 'breath', 'bye', 'cab', 'can', 'cool', 'cre', 'cur', 'daddy', 'detail', 'dickhead', 'do', 'doe', 'ear', 'everyon', 'ex', 'fath', 'feat', 'fin', 'for', 'fuck', 'funny', 'get', 'go', 'god', 'good', 'goodby', 'googl', 'gre', 'hahah', 'headlin', 'hel', 'hello', 'help', 'hi', 'hotel', 'how', 'i', 'ident', 'introduc', 'is', 'it', 'jok', 'kil', 'know', 'konichiw', 'kush', 'lat', 'laugh', 'lenny', 'mak', 'me', 'motherfuck', 'murd', 'myself', 'nam', 'namast', 'new', 'no', 'noth', 'off', 'ok', 'old', 'on', 'reserv', 'restraunt', 'run', 'sci', 'search', 'see', 'sing', 'someth', 'story', 'tabl',

In [3]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word - create base word, in attempt to represent related words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training, dtype = object)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])

In [4]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [5]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [6]:
model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=5, verbose=1)

Epoch 1/100
20/20 [==============================] - 0s 1ms/step - loss: 3.1172 - accuracy: 0.0417
Epoch 2/100
20/20 [==============================] - 0s 1ms/step - loss: 3.0323 - accuracy: 0.1042
Epoch 3/100
20/20 [==============================] - 0s 1ms/step - loss: 2.9638 - accuracy: 0.1458
Epoch 4/100
20/20 [==============================] - 0s 1ms/step - loss: 2.8983 - accuracy: 0.1458
Epoch 5/100
20/20 [==============================] - 0s 926us/step - loss: 2.8483 - accuracy: 0.1458
Epoch 6/100
20/20 [==============================] - 0s 1ms/step - loss: 2.7290 - accuracy: 0.1979
Epoch 7/100
20/20 [==============================] - 0s 828us/step - loss: 2.6557 - accuracy: 0.2604
Epoch 8/100
20/20 [==============================] - 0s 752us/step - loss: 2.4537 - accuracy: 0.3646
Epoch 9/100
20/20 [==============================] - 0s 782us/step - loss: 2.3816 - accuracy: 0.2917
Epoch 10/100
20/20 [==============================] - 0s 1ms/step - loss: 2.3062 - accuracy: 0.3854
E

20/20 [==============================] - 0s 1ms/step - loss: 0.1497 - accuracy: 0.9792
Epoch 83/100
20/20 [==============================] - 0s 1ms/step - loss: 0.1105 - accuracy: 0.9792
Epoch 84/100
20/20 [==============================] - 0s 1ms/step - loss: 0.2022 - accuracy: 0.9271
Epoch 85/100
20/20 [==============================] - 0s 1ms/step - loss: 0.1909 - accuracy: 0.9479
Epoch 86/100
20/20 [==============================] - 0s 1ms/step - loss: 0.1276 - accuracy: 0.9688
Epoch 87/100
20/20 [==============================] - 0s 1ms/step - loss: 0.1316 - accuracy: 0.9688
Epoch 88/100
20/20 [==============================] - 0s 1ms/step - loss: 0.1420 - accuracy: 0.9688
Epoch 89/100
20/20 [==============================] - 0s 1ms/step - loss: 0.1409 - accuracy: 0.9583
Epoch 90/100
20/20 [==============================] - 0s 1ms/step - loss: 0.1707 - accuracy: 0.9375
Epoch 91/100
20/20 [==============================] - 0s 1ms/step - loss: 0.1834 - accuracy: 0.9479
Epoch 92/100


In [7]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

p = bow("Load blood pressure for patient", words)
print (p)
print (classes)

found in bag: for
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0]
['God', 'about me', 'abuse', 'age', 'alive', 'bookings', 'creator', 'fillers', 'goodbye', 'google', 'greeting', 'joke', 'killing', 'lenny', 'myself', 'news', 'stories', 'talk', 'tasks', 'thanks', 'weather', 'wikipedia']


In [8]:

model.save('chatbot.h5',model)
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "learning_chatbot.pkl", "wb" ) )

In [9]:
def classify_local(sentence):
    ERROR_THRESHOLD = 0.25
    
    # generate probabilities from the model
    input_data = pd.DataFrame([bow(sentence, words)], dtype=float, index=['input'])
    results = model.predict([input_data])[0]
    # filter out predictions below a threshold, and provide intent index
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], str(r[1])))
    # return tuple of intent and probability
    
    return return_list